In [1]:
import os
import glob
import pandas as pd
import optopsy as op

In [2]:
# Read option data
df = pd.read_csv("rut_historic.csv")

In [3]:
df.columns

Index(['date', 'symbol', 'exchange', 'company_name', 'stock_price_close',
       'option_symbol', 'option_expiration', 'strike', 'call_put', 'style',
       'open', 'high', 'low', 'close', 'bid', 'ask', 'mean_price',
       'settlement', 'iv', 'volume', 'open_interest', 'stock_price_for_iv',
       'forward_price', 'isinterpolated', 'delta', 'vega', 'gamma', 'theta',
       'rho'],
      dtype='object')

In [10]:
df.iloc[1000020:1000030]

,date,symbol,exchange,company_name,stock_price_close,option_symbol,option_expiration,strike,call_put,style,open,high,low,close,bid,ask,mean_price,settlement,iv,volume,open_interest,stock_price_for_iv,forward_price,isinterpolated,delta,vega,gamma,theta,rho
1000020,09/10/2012,RUT,CBOE,CBOE RUSSELL 2000 INDEX,839.37,RUT 121117C00760000,11/16/2012,760.0,C,E,0.00,0.00,0.00,0.00,82.7,84.7,83.70,0.0,0.225289,0,18,839.37,837.408,NaN,0.851836,0.818287,0.002819,-0.112610,1.154692
1000021,09/10/2012,RUT,CBOE,CBOE RUSSELL 2000 INDEX,839.37,RUT 121117P00760000,11/16/2012,760.0,P,E,7.64,7.94,7.64,7.92,8.3,9.2,8.75,0.0,0.253465,12,578,839.37,837.408,NaN,-0.170730,0.909939,0.002786,-0.177596,-0.278118
1000022,09/10/2012,RUT,CBOE,CBOE RUSSELL 2000 INDEX,839.37,RUT 121117C00770000,11/16/2012,770.0,C,E,0.00,0.00,0.00,0.00,74.3,76.2,75.25,0.0,0.221828,0,34,839.37,837.408,NaN,0.822022,0.924518,0.003234,-0.129081,1.124375
1000023,09/10/2012,RUT,CBOE,CBOE RUSSELL 2000 INDEX,839.37,RUT 121117P00770000,11/16/2012,770.0,P,E,9.24,9.24,9.21,9.21,9.9,10.9,10.40,0.0,0.248244,6,167,839.37,837.408,NaN,-0.198781,1.000139,0.003126,-0.191605,-0.324201
1000024,09/10/2012,RUT,CBOE,CBOE RUSSELL 2000 INDEX,839.37,RUT 121117C00780000,11/16/2012,780.0,C,E,0.00,0.00,0.00,0.00,66.0,67.8,66.90,0.0,0.216356,0,2,839.37,837.408,NaN,0.789905,1.025085,0.003677,-0.142597,1.090340
1000025,09/10/2012,RUT,CBOE,CBOE RUSSELL 2000 INDEX,839.37,RUT 121117P00780000,11/16/2012,780.0,P,E,11.60,11.77,11.58,11.58,11.6,12.7,12.15,0.0,0.241453,3,50,839.37,837.408,NaN,-0.229061,1.086128,0.003491,-0.202928,-0.373889
1000026,09/10/2012,RUT,CBOE,CBOE RUSSELL 2000 INDEX,839.37,RUT 121117C00790000,11/16/2012,790.0,C,E,0.00,0.00,0.00,0.00,58.1,59.9,59.00,0.0,0.212147,0,19,839.37,837.408,NaN,0.752095,1.126901,0.004122,-0.156712,1.046742
1000027,09/10/2012,RUT,CBOE,CBOE RUSSELL 2000 INDEX,839.37,RUT 121117P00790000,11/16/2012,790.0,P,E,12.81,12.81,12.00,12.00,13.7,14.8,14.25,0.0,0.235270,7,80,839.37,837.408,NaN,-0.263613,1.170981,0.003862,-0.213831,-0.430776
1000028,09/10/2012,RUT,CBOE,CBOE RUSSELL 2000 INDEX,839.37,RUT 121117C00800000,11/16/2012,800.0,C,E,54.00,54.05,52.90,53.18,50.6,52.4,51.50,0.0,0.208116,46,336,839.37,837.408,NaN,0.709616,1.221805,0.004556,-0.169404,0.995244
1000029,09/10/2012,RUT,CBOE,CBOE RUSSELL 2000 INDEX,839.37,RUT 121117P00800000,11/16/2012,800.0,P,E,15.00,16.00,14.98,16.00,16.0,17.2,16.60,0.0,0.228464,6,497,839.37,837.408,NaN,-0.301709,1.249316,0.004243,-0.222347,-0.493562


In [13]:
def toOpFormat(df, mapping=None):
    """Pandas DF to Optopsy format

    Args:
        df (pd.DataFrame): Option data
        mapping (dict, optional): Mapping if need to rename columns. {OldName: NewName}.  Defaults to None.
         The following column names are needed:
         "underlying_symbol", "underlying_price", "option_type", "expiration", "quote_date", "strike", "bid", "ask"

    Returns:
        _type_: _description_
    """
    if mapping:
        df = df.rename(columns=mapping)
    df = df[["underlying_symbol", "underlying_price", "option_type", "expiration", "quote_date", "strike", "bid", "ask"]]
    df["expiration"] = pd.to_datetime(df["expiration"])
    df["quote_date"] = pd.to_datetime(df["quote_date"])
    return df

In [14]:
mapping = {
    "symbol": "underlying_symbol", 
    "stock_price_close": "underlying_price",
    "call_put": "option_type",
    "option_expiration": "expiration",
    "date": "quote_date"
}
opdf = toOpFormat(df, mapping)

In [32]:
opdf.iloc[1000000:1000010]

,underlying_symbol,underlying_price,option_type,expiration,quote_date,strike,bid,ask,days_to_expi,moneyness
1000000,RUT,839.37,C,2012-11-16,2012-09-10,675.0,160.90,162.8,67 days,0.195825
1000001,RUT,839.37,P,2012-11-16,2012-09-10,675.0,1.65,2.4,67 days,-0.195825
1000002,RUT,839.37,C,2012-11-16,2012-09-10,680.0,156.10,158.1,67 days,0.189869
1000003,RUT,839.37,P,2012-11-16,2012-09-10,680.0,1.80,2.3,67 days,-0.189869
1000004,RUT,839.37,C,2012-11-16,2012-09-10,685.0,151.30,153.2,67 days,0.183912
1000005,RUT,839.37,P,2012-11-16,2012-09-10,685.0,2.00,2.9,67 days,-0.183912
1000006,RUT,839.37,C,2012-11-16,2012-09-10,690.0,146.40,148.5,67 days,0.177955
1000007,RUT,839.37,P,2012-11-16,2012-09-10,690.0,2.20,3.2,67 days,-0.177955
1000008,RUT,839.37,C,2012-11-16,2012-09-10,700.0,136.90,139.0,67 days,0.166041
1000009,RUT,839.37,P,2012-11-16,2012-09-10,700.0,2.75,3.2,67 days,-0.166041


In [57]:
opdf[(opdf["quote_date"] == "2013-04-02") & (opdf["days_to_expi"] <= pd.Timedelta(days=7))]

,underlying_symbol,underlying_price,option_type,expiration,quote_date,strike,bid,ask,days_to_expi,moneyness
2368422,RUT,934.3,C,2013-04-05,2013-04-02,800.0,134.00,135.70,3 days,0.143744
2368423,RUT,934.3,P,2013-04-05,2013-04-02,800.0,0.00,0.15,3 days,-0.143744
2368424,RUT,934.3,C,2013-04-05,2013-04-02,810.0,124.00,125.70,3 days,0.133041
2368425,RUT,934.3,P,2013-04-05,2013-04-02,810.0,0.00,0.10,3 days,-0.133041
2368426,RUT,934.3,C,2013-04-05,2013-04-02,820.0,114.00,115.70,3 days,0.122338
2368427,RUT,934.3,P,2013-04-05,2013-04-02,820.0,0.00,0.05,3 days,-0.122338
2368428,RUT,934.3,C,2013-04-05,2013-04-02,830.0,104.00,105.70,3 days,0.111634
2368429,RUT,934.3,P,2013-04-05,2013-04-02,830.0,0.00,0.15,3 days,-0.111634
2368430,RUT,934.3,C,2013-04-05,2013-04-02,840.0,94.00,95.70,3 days,0.100931
2368431,RUT,934.3,P,2013-04-05,2013-04-02,840.0,0.00,0.05,3 days,-0.100931


In [52]:
opdf["quote_date"].unique()

<DatetimeArray>
['2010-05-21 00:00:00', '2010-05-24 00:00:00', '2010-05-25 00:00:00',
 '2010-05-26 00:00:00', '2010-05-27 00:00:00', '2010-05-28 00:00:00',
 '2010-06-01 00:00:00', '2010-06-02 00:00:00', '2010-06-03 00:00:00',
 '2010-06-04 00:00:00',
 ...
 '2013-03-19 00:00:00', '2013-03-20 00:00:00', '2013-03-21 00:00:00',
 '2013-03-22 00:00:00', '2013-03-25 00:00:00', '2013-03-26 00:00:00',
 '2013-03-27 00:00:00', '2013-03-28 00:00:00', '2013-04-01 00:00:00',
 '2013-04-02 00:00:00']
Length: 2040, dtype: datetime64[ns]

In [ ]:
# Also possible to read from csv directly with Optopsy to handle data formating. 
# Need to provide column index (starting by 0 for first col) for each metric.
rut_chains = op.csv_data("rut_historic.csv", underlying_symbol=1, underlying_price=4, option_type=8,
                         expiration=6, quote_date=0, strike=7, bid=14, ask=15)

## Strategies

- **dte_range**: The range of days remaining until the options expire. For example, (0, 7] includes all options that will expire in 0 to 7 days based on the quote date.
- **otm_pct_range**: Range of how far out of the money the options are, expressed as a percentage. For example, (-0.5, -0.45] means the options are between 50% and 45% out of the money.
- **count**: The number of options contracts that fall within the specified dte_range and otm_pct_range. For instance, there are 666 options contracts that expire in 0 to 7 days and are 50% to 45% out of the money.
- **mean**: This is the average percent change of the options within the specified dte_range and otm_pct_range.

In [22]:
kwargs = {
    "dte_interval": 30,
    "max_entry_dte": 91,
    "exit_dte": 0,
    "otm_pct_interval": 0.05,
    "max_otm_pct": 0.1,
    "min_bid_ask": 0.05,
    "drop_nan": True,
    "raw": False,
}

In [23]:
# Short calls strategy
op.short_calls(opdf, **kwargs).round(2)

,dte_range,otm_pct_range,count,mean,std,min,25%,50%,75%,max
0,"(0, 30]","(-0.1, -0.05]",13965.0,-0.96,0.19,-1.0,-1.0,-1.0,-1.0,1.13
1,"(0, 30]","(-0.05, 0.0]",20653.0,-0.96,0.23,-1.0,-1.0,-1.0,-1.0,3.61
2,"(0, 30]","(0.0, 0.05]",18049.0,-0.97,0.40,-1.0,-1.0,-1.0,-1.0,19.00
3,"(30, 60]","(-0.1, -0.05]",5217.0,-0.97,0.18,-1.0,-1.0,-1.0,-1.0,0.91
4,"(30, 60]","(-0.05, 0.0]",9056.0,-0.94,0.34,-1.0,-1.0,-1.0,-1.0,3.02
5,"(30, 60]","(0.0, 0.05]",10548.0,-0.94,0.46,-1.0,-1.0,-1.0,-1.0,9.00
6,"(60, 90]","(-0.1, -0.05]",2377.0,-0.96,0.18,-1.0,-1.0,-1.0,-1.0,0.63
7,"(60, 90]","(-0.05, 0.0]",4284.0,-0.96,0.24,-1.0,-1.0,-1.0,-1.0,1.13
8,"(60, 90]","(0.0, 0.05]",5615.0,-0.97,0.19,-1.0,-1.0,-1.0,-1.0,1.19


In [40]:
filtered = opdf.loc[(opdf["days_to_expi"] > pd.Timedelta(days=0)) & (opdf["days_to_expi"] <= pd.Timedelta(days=30)) & (opdf["option_type"] == "C") & (opdf["moneyness"] > - 0.1) & (opdf["moneyness"] <= - 0.05)]

In [49]:
len(filtered[["option_type", "expiration", "strike"]].drop_duplicates())

2684

In [24]:
# Long calls strategy
op.long_calls(opdf, **kwargs).round(2)

,dte_range,otm_pct_range,count,mean,std,min,25%,50%,75%,max
0,"(0, 30]","(-0.1, -0.05]",13965.0,-0.96,0.19,-1.0,-1.0,-1.0,-1.0,1.13
1,"(0, 30]","(-0.05, 0.0]",20653.0,-0.96,0.23,-1.0,-1.0,-1.0,-1.0,3.61
2,"(0, 30]","(0.0, 0.05]",18049.0,-0.97,0.40,-1.0,-1.0,-1.0,-1.0,19.00
3,"(30, 60]","(-0.1, -0.05]",5217.0,-0.97,0.18,-1.0,-1.0,-1.0,-1.0,0.91
4,"(30, 60]","(-0.05, 0.0]",9056.0,-0.94,0.34,-1.0,-1.0,-1.0,-1.0,3.02
5,"(30, 60]","(0.0, 0.05]",10548.0,-0.94,0.46,-1.0,-1.0,-1.0,-1.0,9.00
6,"(60, 90]","(-0.1, -0.05]",2377.0,-0.96,0.18,-1.0,-1.0,-1.0,-1.0,0.63
7,"(60, 90]","(-0.05, 0.0]",4284.0,-0.96,0.24,-1.0,-1.0,-1.0,-1.0,1.13
8,"(60, 90]","(0.0, 0.05]",5615.0,-0.97,0.19,-1.0,-1.0,-1.0,-1.0,1.19


In [45]:
kwargs = {
    "otm_pct_interval": 0.01,
    "max_otm_pct": 0.02,
}
# Long straddle strategy
op.short_straddles(opdf, **kwargs).round(2)

,dte_range,otm_pct_range,count,mean,std,min,25%,50%,75%,max
0,"(0, 7]","(-0.02, -0.01]",984.0,0.96,0.19,-0.63,1.0,1.0,1.0,1.0
1,"(0, 7]","(-0.01, 0.0]",1167.0,0.95,0.21,-0.76,1.0,1.0,1.0,1.0
2,"(0, 7]","(0.0, 0.01]",1069.0,0.95,0.23,-1.07,1.0,1.0,1.0,1.0
3,"(7, 14]","(-0.02, -0.01]",509.0,0.98,0.13,-0.10,1.0,1.0,1.0,1.0
4,"(7, 14]","(-0.01, 0.0]",615.0,0.98,0.15,-0.80,1.0,1.0,1.0,1.0
5,"(7, 14]","(0.0, 0.01]",646.0,0.99,0.10,-0.97,1.0,1.0,1.0,1.0
6,"(14, 21]","(-0.02, -0.01]",353.0,0.99,0.05,0.38,1.0,1.0,1.0,1.0
7,"(14, 21]","(-0.01, 0.0]",407.0,0.99,0.07,0.33,1.0,1.0,1.0,1.0
8,"(14, 21]","(0.0, 0.01]",416.0,0.99,0.07,0.28,1.0,1.0,1.0,1.0
9,"(21, 28]","(-0.02, -0.01]",268.0,1.00,0.01,0.82,1.0,1.0,1.0,1.0


Let’s analyze the performance of all short strangles that expire within 60 days, with a max days to expiration of 70, exit at 10 days to expiration, and each leg no further than 10% out of the money.

In [26]:
# Short strangles
# Leg 1 is put
op.short_strangles(
    rut_chains, 
    dte_interval=60, 
    max_entry_dte=70, 
    exit_dte=10,
    otm_pct_interval=0.01,
    max_otm_pct=0.10
).round(2)

,dte_range,otm_pct_range_leg1,otm_pct_range_leg2,count,mean,std,min,25%,50%,75%,max
0,"(0, 60]","(-0.1, -0.09]","(-0.1, -0.09]",398.0,0.24,0.21,-0.12,0.07,0.21,0.39,0.72
1,"(0, 60]","(-0.1, -0.09]","(-0.09, -0.08]",2602.0,0.23,0.23,-0.22,0.04,0.21,0.40,0.77
2,"(0, 60]","(-0.1, -0.09]","(-0.08, -0.07]",2857.0,0.26,0.25,-0.26,0.05,0.24,0.45,0.81
3,"(0, 60]","(-0.1, -0.09]","(-0.07, -0.06]",3023.0,0.29,0.27,-0.28,0.07,0.27,0.49,0.85
4,"(0, 60]","(-0.1, -0.09]","(-0.06, -0.05]",2941.0,0.32,0.28,-0.33,0.08,0.30,0.54,0.88
5,"(0, 60]","(-0.1, -0.09]","(-0.05, -0.04]",2988.0,0.35,0.29,-0.37,0.11,0.35,0.60,0.90
6,"(0, 60]","(-0.1, -0.09]","(-0.04, -0.03]",3070.0,0.39,0.31,-0.48,0.14,0.41,0.65,0.92
7,"(0, 60]","(-0.1, -0.09]","(-0.03, -0.02]",3218.0,0.44,0.32,-0.57,0.21,0.49,0.72,0.93
8,"(0, 60]","(-0.1, -0.09]","(-0.02, -0.01]",3145.0,0.48,0.33,-0.69,0.25,0.54,0.76,0.93
9,"(0, 60]","(-0.1, -0.09]","(-0.01, -0.0]",3254.0,0.53,0.32,-0.88,0.35,0.61,0.79,0.94
